In [ ]:
pip install -q --upgrade langchain langchain-openai langchain-core langchain_community langchain_chroma sentence_transformers docx2txt pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.6 MB/s eta 

In [ ]:
import langchain
print(langchain.__version__)

1.1.2


In [1]:
import os

from google.colab import userdata
open_ai_key = os.getenv("OPENAI_API_KEY") or userdata.get("OPENAI_API_KEY")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY") or userdata.get("LANGCHAIN_API_KEY")

#print(open_ai_key)
#print(langchain_api_key)


In [ ]:
os.environ["OPENAI_API_KEY"] = open_ai_key

os.environ["LANGCHAIN_TRACING_V2"] = 'true'
os.environ["LANGCHAIN_PROJECT"] = 'langchain-rag'
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tracers.context import tracing_v2_enabled

# with tracing_v2_enabled():
#   llm_resonse = llm.invoke(query_to_llm)

In [ ]:
from langchain_community.document_loaders import PyPDFLoader, TextLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

from typing import List

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, length_function=len)


# pdf_loader = PyPDFLoader("/content/documents/Generative AI.pdf")
# documents = pdf_loader.load()

# splits = text_splitter.split_documents(documents)

# print(f'Document splitted into {len(splits)} chunks')

In [ ]:
# print(splits[0])

In [ ]:
# print(splits[0].metadata)

In [ ]:
# print(splits[0].page_content)

In [ ]:
# Split / Chunking

# function to fetch all files from documents folder
def fetch_files(path: str) -> List[Document]:
  documents = []
  for file_name in os.listdir(path):
    file_path = os.path.join(path, file_name)
    if file_name.endswith(".pdf"):
      loader = PyPDFLoader(file_path)
    elif file_name.endswith(".docx"):
      loader = Docx2txtLoader(file_path)
    else:
      print(f'Unsupported file type: {file_name}')
      continue
    documents.extend(loader.load())
  return documents


path = '/content/documents'
documents = fetch_files(path)

print(f'Loaded {len(documents)} documents from folder.')

splits = text_splitter.split_documents(documents)

print(f'Document splitted into {len(splits)} chunks')

Loaded 41 documents from folder.
Document splitted into 141 chunks


In [ ]:
# Embedding

from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
document_embeddings = embedding_function.embed_documents([split.page_content for split in splits])

document_embeddings[0]

/tmp/ipython-input-1868341026.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[-0.11065839976072311,
 0.00826304405927658,
 0.036357101052999496,
 0.0881635919213295,
 -0.048763457685709,
 0.005361645948141813,
 -0.029269825667142868,
 0.03008352220058441,
 -0.10546578466892242,
 0.012956855818629265,
 -0.057093046605587006,
 0.001984980423003435,
 0.057253818958997726,
 0.023385601118206978,
 0.046851277351379395,
 0.10288745164871216,
 -0.005931116174906492,
 -0.06867562234401703,
 -0.029753856360912323,
 -0.0856204703450203,
 0.07045819610357285,
 0.028267877176404,
 0.05100534111261368,
 0.10901660472154617,
 -0.0026733707636594772,
 0.013336803764104843,
 -0.0259296502918005,
 -0.0005499112303368747,
 0.03338845819234848,
 -0.05799724906682968,
 0.049147460609674454,
 0.002760153729468584,
 0.04723860323429108,
 0.004487505182623863,
 0.04739215970039368,
 0.026645733043551445,
 0.02614787593483925,
 -0.023769047111272812,
 0.04589219018816948,
 0.023693358525633812,
 -0.056342653930187225,
 -0.05870335176587105,
 -0.09085451811552048,
 -0.03211691975593567

In [ ]:
# # Store in Vector DB

# from langchain_chroma import Chroma

# openai_embedding = OpenAIEmbeddings()
# collection_name = 'apple_hig_documents'
# vector_store = Chroma.from_documents(documents=splits, embedding=openai_embedding, collection_name=collection_name)

# print('Vector store created and persisted to "./chroma_db"')

In [ ]:
pip install sentence-transformers


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

persist_dir = 'chroma_db'
vector_store = Chroma.from_documents(
    documents=splits,
    embedding=embedding_model,
    collection_name='apple_hig_documents',
    persist_directory=persist_dir
)
print('Vector store created and persisted to "chroma_db"')

Vector store created and persisted to "chroma_db"


In [ ]:
# Perform similarty search

quey = 'You can position toolbar items in which three locations?'

seatch_results = vector_store.similarity_search(quey, k=4)

print(f'\nTop 4 most relevant chunks for the query: "{quey}"\n')
for index, result in enumerate(seatch_results):
  print(f'Result {index}')
  print(f'Source: {result.metadata.get('source', 'Unknown')}')
  print(f'Content: {result.page_content}')

  print()


Top 4 most relevant chunks for the query: "You can position toolbar items in which three locations?"

Result 0
Source: /content/documents/Toolbars.pdf
Content: To position items in the groupings you want, pin them to the leading edge, center, or trailing
edge, and insert space between buttons or other items where appropriate.
Group toolbar items logically by function and frequency of use. For example, Keynote
includes several sections that are based on functionality, including one for presentation-level
commands, one for playback commands, and one for object insertion.
Group navigation controls and critical actions like Done, Close, or Save in dedicated, familiar,
and visually distinct sections. This reflects their importance and helps people discover and
understand these actions.
Item groupings
10/12/2025, 22:59 Toolbars | Apple Developer Documentation
https://developer.apple.com/design/human-interface-guidelines/toolbars 4/9

Result 1
Source: /content/documents/Toolbars.pdf
Content:

In [ ]:
from re import search
retriever = vector_store.as_retriever(search_kwargs={'k': 4})
retriever.invoke('You can position toolbar items in which three locations?')

[Document(id='a4516d94-9a1b-48b5-a4f0-93e2d03163ac', metadata={'page_label': '4', 'producer': 'Skia/PDF m143', 'total_pages': 9, 'page': 3, 'source': '/content/documents/Toolbars.pdf', 'creationdate': '2025-12-10T17:29:09+00:00', 'moddate': '2025-12-10T17:29:09+00:00', 'creator': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36', 'title': 'Toolbars | Apple Developer Documentation'}, page_content='To position items in the groupings you want, pin them to the leading edge, center, or trailing\nedge, and insert space between buttons or other items where appropriate.\nGroup toolbar items logically by function and frequency of use. For example, Keynote\nincludes several sections that are based on functionality, including one for presentation-level\ncommands, one for playback commands, and one for object insertion.\nGroup navigation controls and critical actions like Done, Close, or Save in dedicated, familiar,\nand visuall

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
template = """
Answer the question based only on the following context:
{context}

Question: {question}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | prompt
    #| ChatOpenAI(temperature=0)
)
rag_chain.invoke('You can position toolbar items in which three locations?')

ChatPromptValue(messages=[HumanMessage(content="\nAnswer the question based only on the following context:\n[Document(id='a4516d94-9a1b-48b5-a4f0-93e2d03163ac', metadata={'moddate': '2025-12-10T17:29:09+00:00', 'total_pages': 9, 'page': 3, 'creationdate': '2025-12-10T17:29:09+00:00', 'creator': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36', 'producer': 'Skia/PDF m143', 'title': 'Toolbars | Apple Developer Documentation', 'page_label': '4', 'source': '/content/documents/Toolbars.pdf'}, page_content='To position items in the groupings you want, pin them to the leading edge, center, or trailing\\nedge, and insert space between buttons or other items where appropriate.\\nGroup toolbar items logically by function and frequency of use. For example, Keynote\\nincludes several sections that are based on functionality, including one for presentation-level\\ncommands, one for playback commands, and one for object insertion

In [ ]:
def doctToStr(docs):
  return '\n\n'.join(doc.page_content for doc in docs)

In [ ]:
rag_chain = (
    {'context': retriever | doctToStr, 'question': RunnablePassthrough()}
    | prompt
)
rag_chain.invoke('You can position toolbar items in which three locations?')

ChatPromptValue(messages=[HumanMessage(content='\nAnswer the question based only on the following context:\nTo position items in the groupings you want, pin them to the leading edge, center, or trailing\nedge, and insert space between buttons or other items where appropriate.\nGroup toolbar items logically by function and frequency of use. For example, Keynote\nincludes several sections that are based on functionality, including one for presentation-level\ncommands, one for playback commands, and one for object insertion.\nGroup navigation controls and critical actions like Done, Close, or Save in dedicated, familiar,\nand visually distinct sections. This reflects their importance and helps people discover and\nunderstand these actions.\nItem groupings\n10/12/2025, 22:59 Toolbars | Apple Developer Documentation\nhttps://developer.apple.com/design/human-interface-guidelines/toolbars 4/9\n\nitems are always available, items on the toolbarʼs leading edge arenʼt customizable.\nCenter area.

In [ ]:
# parsing output
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
#output_parser.invoke(llm_resonse)


In [ ]:
llm = ChatOpenAI(model_name="gpt-4o-mini")

In [ ]:
rag_chain = (
    {'context': retriever | doctToStr, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
question = 'You can position toolbar items in which three locations?'
response = rag_chain.invoke(question)
print(f'Question: {question}\n')
print(f'Answer: {response}')


Question: You can position toolbar items in which three locations?

Answer: You can position toolbar items in the leading edge, center area, and trailing edge.
